## AI Assistant Development and Experimentation Notebook

### Tools used: Pushover, OpenAI, Gradio, Python, PyPDF, uv Package Manager

In [1]:
# import

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import json
import os
import requests

In [2]:
# secrets .env

load_dotenv(override=True)

openai = OpenAI()

In [3]:
# Setting up Pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [6]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [7]:
push("Testing 1 2 3!")

Push: Testing 1 2 3!


### Function as tools for agent

In [9]:

def record_user_details(email, name="Name not provided", notes="Not provided"):
    push(f"User details: Recording interest from {name}, with email {email}, and notes {notes}")
    return {"recorded": "ok"}

def record_unknown_question(question):
    push(f"Recording unknown question that I don't know the answer to: {question}")
    return {"recorded": "ok"}